In [1]:
import pandas as pd
from IPython.display import display
from nltk.corpus import stopwords

from capstone_project import preprocessor

%matplotlib inline

In [2]:
#the current working directory for python is the capstone_project/notebook folder
data = pd.read_pickle("../output/data/train_data.pkl")
data.fillna("", inplace=True)

In [ ]:
import spacy
nlp = spacy.load("en")
test = nlp(unicode("I'll do that. This is better.", "utf-8"))
for token in test:
    print token, token.lemma, token.lemma_

In [ ]:
x, y = data.iloc[:, 3:-1].values, data.iloc[:, -1].values
print type(y), y[:10]
test = data['is_duplicate'].values
print type(test), test[:10]

In [ ]:
#data[data.isnull().any(axis=1)]


In [ ]:
#data.add(data[["question1","question2"]].applymap(preprocess.tokenize))

In [ ]:
new_data = preprocessor.create_features(data)
display(new_data.head(2))

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
'The sun is shining',
'The weather is sweet',
'The sun is shining and the weather is sweet'])
#bag = count.fit_transform(docs)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp

tfidf1 = TfidfVectorizer(tokenizer=preprocessor.tokenize)
#test = tfidf.fit_transform(data["question1"][:10])
X = tfidf1.fit_transform(docs)

In [ ]:
print X.toarray()

In [ ]:
#print test, "\n", test.toarray()[0]
#print sp.sparse.hstack(docs)
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X,y=[0,0,1])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None,
                                tokenizer=preprocessor.tokenize)
df = data

tfidf.fit(pd.concat([df['question1'], df['question2']]))

tfidfs_q1 = tfidf.transform(df['question1'])
tfidfs_q2 = tfidf.transform(df['question2'])

# Stack horizontally since the words in q1 and q2 should be arranged in separate columns
tfidf_q1_q2 = hstack([tfidfs_q1, tfidfs_q2])

#clf = LogisticRegression()
#clf.fit(X,y=data.iloc[:, -1].values)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = preprocessor.TfidfTransformer()
clf = LogisticRegression()

y = data["is_duplicate"].values

pipe = Pipeline([('tfidf', tfidf), ('clf', clf)])

pipe.fit(data, y)

/home/andre/software/anaconda3/envs/capstone_project/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


<363915x62463 sparse matrix of type '<type 'numpy.float64'>'
	with 2915884 stored elements in Compressed Sparse Column format>

In [17]:
pipe.steps[0]
#pipe.get_params()

('tfidf', TfidfTransformer())

In [38]:
test = pipe.transform(data.iloc[0:2]).toarray()
test2 = pipe.predict(data.iloc[0:10])
print test, "\n", test2

pipe2 = Pipeline(pipe.steps[:-1])
test2 = pipe2.transform(data.iloc[0:3]).toarray()
print pipe2, test2

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] 
[1 1 0 0 1 0 1 0 0 0]
Pipeline(steps=[('tfidf', TfidfTransformer())]) [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


/home/andre/software/anaconda3/envs/capstone_project/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
